# Importing libraies

In [ ]:
import pandas as pd
import numpy as np
import itertools
import psycopg2
import datetime as dt
import pymysql
import time

import seaborn as sns
# import Functions_Transform_Data as ftd

# Connect database

In [ ]:
database = "analyse_customer"
user = "postgres"
password = "12345"
host = "localhost"
port = "5432"

In [ ]:
# Connecting data localhost
try:
    connect = psycopg2.connect(database = database, user = user,
                            password = password, host = host, port=port)
    print ("Connected!")
except:
    print ("I am unable to connect to the database")

# Load data

In [ ]:
df_tran = pd.read_sql('SELECT * FROM transaction_info', connect)

In [ ]:
df_cmp_inv = pd.read_csv("../compare_number_of_invoices_of_two_products.csv")
df_cmp_inv.columns.str.match('Unnamed')
df_cmp_inv = df_cmp_inv.loc[:, ~df_cmp_inv.columns.str.match('Unnamed')]

In [ ]:
df_cmp_inv = df_cmp_inv[['customerid','class']]
print(len(df_cmp_inv))
df_cmp_inv.head()

# Deal the customer returned goods

In [ ]:
df_all = df_tran.groupby( ["customerid"] )['sales'].sum().reset_index(name = 'sales')

In [ ]:
ams = df_all[df_all['sales']<0]
print(len(ams))
ls_ams = list(ams.customerid)

In [ ]:
df_tran_re = df_tran[~df_tran.customerid.isin(ls_ams)]
print(len(df_tran_re))
print(len(df_tran) - len(df_tran_re))
df_tran_re.head(2)

In [ ]:
df_tran_re.describe(include='all')

In [ ]:
df_tran_re.dtypes

# Functions

#### 1. Function: Total return product times

In [12]:
def total_return_product_times(df_cmp_inv, df_tran_re, prod_id1, prod_id2):
    """
    Total returned product times.

    The function calculates the ratio of total returned product times of 2 products for each customer.

    Parameters
    ----------
    arg1 : DataFrame
        List of customers
    arg2 : DateFrame
        Transactions in the category
    arg3 : String
        Product ID 1
    arg4 : String
        Product ID 2   
    Returns
    -------
    DataFrame
        Returns the DataFrame has been added @column that is the ratio of total returned product times of 2 products for each customer.
        @column: RT_return
    """
    
    # PRODUCT 1
    # Count returned product times
    df_tran_re_prod1 = df_tran_re[(df_tran_re['quantity'] < 0) & (df_tran_re['productid'] == prod_id1)]
    df_tran_prod1 = df_tran_re_prod1.groupby(['customerid'], as_index=False)['invoiceno'].count()
    df_tran_prod1 = df_tran_prod1[df_tran_prod1['customerid'] != 'GUEST']
    
    # PRODUCT 2
    # Count returned product times
    df_tran_re_prod2 = df_tran_re[(df_tran_re['quantity'] < 0) & (df_tran_re['productid'] == prod_id2)]
    df_tran_prod2 = df_tran_re_prod2.groupby(['customerid'], as_index=False)['invoiceno'].count()
    df_tran_prod2 = df_tran_prod2[df_tran_prod2['customerid'] != 'GUEST']
    
    # Merge two DataFrame product 1 and product 2
    df_res_re = pd.merge(df_tran_prod1, df_tran_prod2, how='outer', on= 'customerid')
    df_res_re.fillna(0, inplace=True)
    
    # Binning data
    '''
    RL: 0-49%
    RE: 50%
    RG: 51-100%   
    
    '''
    df_res_re['RT_return'] = (df_res_re['invoiceno_x']*100/(df_res_re['invoiceno_x'] + df_res_re['invoiceno_y'])).astype(int)
    df_res_re.replace({'RT_return' : range(0,50)}, 'RL',inplace= True)
    df_res_re.replace({'RT_return' : 50}, 'RE', inplace= True)
    df_res_re.replace({'RT_return' : range(51,101)}, 'RG', inplace= True)

    df_res_re = df_res_re[['customerid','RT_return']]
    
    #Convert customerid to type int
    df_cmp_inv['customerid']=df_cmp_inv['customerid'].astype(int)
    df_res_re['customerid']=df_res_re['customerid'].astype(int)

    #Added @column that is ratio of total returned product times of 2 products for each customer.
    df_data = pd.merge(df_cmp_inv, df_res_re, how = 'left', on = 'customerid')
    
    #replace the NaN to RE which the ratio of 2 products are equal.
    df_data.fillna('RE', inplace=True)
    
    return df_data

#### 2. Function: Quantity purchased in the most recent period

In [13]:
def quantity_purchased(df_cmp_inv, df_tran_re, prod_id1, prod_id2, time = '3M'):
    """

    The function calculates the ratio of quantity purchased in the most recent period of 2 products for each customer.

    Parameters
    ----------
    arg1 : DataFrame
        DataFrame of Customer
    arg2 : DateFrame
        DataFrame of Transaction.
    arg3 : String
        The most recent period
        default: time = '3M'  ---3 Month
        Eg: "1D","1M","1Y"
    arg4 : String
        Product ID 1
    arg5 : String
        Product ID 2
        
    Returns
    -------
    DataFrame
        Returns the DataFrame has been added @column that is the ratio of quantity purchased in the most recent period of 2 products column.
        @column: RT_quantity_LT

    """
    #Get the last time
    df_tran_re.index = df_tran_re['invoicedate']
    df_tran_re = df_tran_re.last(time)
    
    # quantity purchased of product 1
    df_tran_re_prod1 = df_tran_re[(df_tran_re['quantity'] > 0) & (df_tran_re['productid'] == prod_id1)]
    df_tran_re_prod1 = df_tran_re_prod1[df_tran_re_prod1['customerid'] != 'GUEST']
    df_tran_prod1  = df_tran_re_prod1.groupby(['customerid'], as_index=False)['quantity'].sum()
    
    # quantity purchased of product 2
    df_tran_re_prod2 = df_tran_re[(df_tran_re['quantity'] > 0) & (df_tran_re['productid'] == prod_id2)]
    df_tran_re_prod2 = df_tran_re_prod2[df_tran_re_prod2['customerid'] != 'GUEST']
    df_tran_prod2  = df_tran_re_prod2.groupby(['customerid'], as_index=False)['quantity'].sum()
    
    # merge two DataFrame product 1 and product 2
    df_res_qa = pd.merge(df_tran_prod1, df_tran_prod2, how='outer', on= 'customerid')
    df_res_qa.fillna(0, inplace=True)
    
    # Binning data
    '''
    Q1: 0-19%
    Q2: 20-39%
    Q3: 40-59%
    Q4: 60-79%
    Q5: 80-100%
    
    '''
    df_res_qa['RT_quantity_LT'] = (df_res_qa['quantity_x']*100/(df_res_qa['quantity_x'] + df_res_qa['quantity_y'])).astype(int)
    df_res_qa.replace({'RT_quantity_LT' : range(20)}, 'Q1',inplace= True)
    df_res_qa.replace({'RT_quantity_LT' : range(20,40)}, 'Q2', inplace= True)
    df_res_qa.replace({'RT_quantity_LT' : range(40,60)}, 'Q3', inplace= True)
    df_res_qa.replace({'RT_quantity_LT' : range(60,80)}, 'Q4', inplace= True)
    df_res_qa.replace({'RT_quantity_LT' : range(80,101)}, 'Q5', inplace= True)
    
    df_res_qa = df_res_qa[['customerid','RT_quantity_LT']]
    
    
    
    #Convert customerid to type int
    df_cmp_inv['customerid']=df_cmp_inv['customerid'].astype(int)
    df_res_qa['customerid']=df_res_qa['customerid'].astype(int)

    #Added @column that is the ratio of quantity purchased in the most recent period of 2 products column.
    df_data = pd.merge(df_cmp_inv, df_res_qa, how = 'left', on = 'customerid')

    #replace the NaN to Q3 which the ratio of 2 products are equal.
    df_data.fillna('Q3', inplace=True)
    
    return df_data

#### 3. Function: Number of days since last purchase.

In [14]:
def recenvy(df_cmp_inv, df_tran_re, prod_id1, prod_id2, date):
    """
    Number of days since last purchase.

    The function calculates the ratio of number of days since last purchase of 2 products for each customer.

    Parameters
    ----------
    arg1 : DataFrame
        DataFrame of Customer
    arg2 : DateFrame
        DataFrame of Transaction.
    arg3 : String
        Product ID 1
    arg4 : String
        Product ID 2 
    arg5 : np.datetime64
        The present time.
        Eg: '2011-12-10'      
    Returns
    -------
    DataFrame
        Returns the DataFrame has been added @column that is the ratio of number of days since last purchase of 2 products for each customer.
        @column: Recency

    """
    # PRODUCT 1
    # order by time descending
    df_tran_re_prod1 = df_tran_re[(df_tran_re['quantity'] > 0) & (df_tran_re['productid'] == prod_id1)]
    df_tran_prod1 = df_tran_re_prod1.sort_values(by='invoicedate', ascending=False)
    df_tran_prod1 = df_tran_prod1[df_tran_prod1['customerid'] != 'GUEST']
    
    #Group by customerid to find the nearest purchase time for each customer
    df_tran_prod1 = df_tran_prod1.groupby(['customerid'], as_index=False)['invoicedate'].max()
    df_tran_prod1 = df_tran_prod1[['customerid','invoicedate']]
    
    #Calculate the number of days since last purchase
    df_tran_prod1['num_days'] =(date - df_tran_prod1['invoicedate']).dt.days
    df_tran_prod1 = df_tran_prod1[['customerid','num_days']]
    #-----------
    
    # PRODUCT 2
    # order by time descending
    df_tran_re_prod2 = df_tran_re[(df_tran_re['quantity'] > 0) & (df_tran_re['productid'] == prod_id2)]
    df_tran_prod2 = df_tran_re_prod2.sort_values(by='invoicedate', ascending=False)
    df_tran_prod2 = df_tran_prod2[df_tran_prod2['customerid'] != 'GUEST']
    
    #Group by customerid to find the nearest purchase time for each customer
    df_tran_prod2 = df_tran_prod2.groupby(['customerid'], as_index=False)['invoicedate'].max()
    df_tran_prod2 = df_tran_prod2[['customerid','invoicedate']]
    
    #Calculate the number of days since last purchase
    df_tran_prod2['num_days'] =(date - df_tran_prod2['invoicedate']).dt.days
    df_tran_prod2 = df_tran_prod2[['customerid','num_days']]
    #------------
    
    # merge two DataFrame product 1 and product 2
    df_res_rec = pd.merge(df_tran_prod1, df_tran_prod2, how='outer', on= 'customerid')
    df_res_rec.fillna(0, inplace=True)
    
    # Binning data
    '''
    R1: 0-49%
    R2: 50%
    R3: 51-100%

    '''
    df_res_rec['Recency'] = (df_res_rec['num_days_x']*100/(df_res_rec['num_days_x'] + df_res_rec['num_days_y'])).astype(int)
    df_res_rec.replace({'Recency' : range(0,50)}, 'R1',inplace= True)
    df_res_rec.replace({'Recency' : 50}, 'R2', inplace= True)
    df_res_rec.replace({'Recency' : range(51,101)}, 'R3', inplace= True)
    
    df_res_rec = df_res_rec[['customerid','Recency']]
    
    #Convert customerid to type int
    df_cmp_inv['customerid']=df_cmp_inv['customerid'].astype(int)
    df_res_rec['customerid']=df_res_rec['customerid'].astype(int)

    #Added @column that is the ratio of number of days since last purchase of 2 products for each customer.
    df_data = pd.merge(df_cmp_inv, df_res_rec, how = 'left', on = 'customerid')

    #replace the NaN to R2 which the ratio of 2 products are equal.
    df_data.fillna('R2', inplace=True)
    
    return df_data

#### 4. Function: Average budget for specific category in the last k months

In [15]:
df_trans = pd.read_sql('SELECT transaction_info.*, product.category FROM transaction_info JOIN product ON product.productid = transaction_info.productid WHERE category = \'Dining and Entertaining\' AND sales > 0 AND customerid != \'GUEST\'' , connect)

In [100]:
def avg_bud_for_cat_in_k_months(df_tran, df_cmp_inv, month = 3):
    """
    Average budget for specific category in the last k months

    Parameters
    ----------
    df_tran : DataFrame
        Transactions in the category
    df_cmp_inv : DateFrame
        Customers with label
    month : Integer
        Last k months 

    Returns
    -------
    DataFrame
        List average budget of each customers for that category

    """
    
    #Transactions in the last k months
    df_invoices_k_months = df_tran.sort_values(by="invoicedate",ascending=True).set_index("invoicedate").last(str(month) + "M")
    
    #Average budget per invoice in that time
    df_avg_bud_per_inv = df_invoices_k_months.groupby( ["customerid"] )['sales'].mean().reset_index(name = 'sales')
   
    #Separate class for new feature
    result_label = 'avg_bud_' + str(month) + '_months'
    df_avg_bud_per_inv[result_label] = 'Poor'
    df_avg_bud_per_inv.loc[df_avg_bud_per_inv['sales'] > 70, result_label] = 'Rich'
    
    ##Merge result above with labelled customers list 
    df_cmp_inv['customerid'] = df_cmp_inv['customerid'].astype(int)
    df_avg_bud_per_inv['customerid'] = df_avg_bud_per_inv['customerid'].astype(int)
    df_avg_bud_per_inv.drop('sales', inplace = True, axis = 1)
    df_data = pd.merge(df_cmp_inv, df_avg_bud_per_inv, how = 'left', on = 'customerid')
    
    df_data.fillna('Poor', inplace=True)
    
    return df_data

#### 5. Function: Number of invoices in the last k months

In [58]:
def num_invoices_last_k_months(df_tran, df_cmp_inv, prod_id1, prod_id2, month):
    """
    Number of invoices in the last k months

    Parameters
     ----------
    df_tran : DataFrame
        Transactions in the category
    df_cmp_inv : DateFrame
        Customers with label
    prod_id1 : String
        Product ID 1
    prod_id2 : String
        Product ID 2   
    month : Integer
        Last k months 

    Returns
    -------
    DataFrame
        List compare number of invoices of each customer for each product in the last k months

    """
    #Transactions in the last k months
    df_invoices_k_months = df_tran[(df_tran['customerid'] != 'GUEST')] \
                            .sort_values(by="invoicedate",ascending=True).set_index("invoicedate") \
                            .last(str(month) + "M")
    
    #Number of invoices for product 1
    df_num_invoices_prod1 = df_invoices_k_months[(df_invoices_k_months['productid'] == prod_id1)] \
                            .groupby( ["customerid"] )['invoiceno'] \
                            .count() \
                            .reset_index(name = 'invoiceno')
    
    #Number of invoices for product 2
    df_num_invoices_prod2 = df_invoices_k_months[(df_invoices_k_months['productid'] == prod_id2)] \
                            .groupby( ["customerid"] )['invoiceno'] \
                            .count() \
                            .reset_index(name = 'invoiceno')
    
    #Merge 2 list above to compare
    df_result = pd.merge(df_num_invoices_prod1, df_num_invoices_prod2, how='outer', on= 'customerid')
    df_result.fillna(0, inplace=True)
    
    #Add new feature into result
    result_label = 'Num_inv_' + str(month) + '_months'
    df_result[result_label] = (df_result['invoiceno_x']*100/(df_result['invoiceno_x'] + df_result['invoiceno_y'])).astype(int)
    
    #Separate class for new feature
    df_result.replace({result_label : range(0, 20)}, 'N1',inplace= True)
    df_result.replace({result_label : range(20, 40)}, 'N2', inplace= True)
    df_result.replace({result_label : range(40, 60)}, 'N3', inplace= True)
    df_result.replace({result_label : range(60, 80)}, 'N4', inplace= True)
    df_result.replace({result_label : range(80, 101)}, 'N5', inplace= True)
    
    #Merge result above with labelled customers list 
    df_result = df_result[['customerid', result_label]]
    df_cmp_inv['customerid'] = df_cmp_inv['customerid'].astype(int)
    df_result['customerid'] = df_result['customerid'].astype(int)
    df_data = pd.merge(df_cmp_inv, df_result, how = 'left', on = 'customerid')
    #Fill NA value with class N3
    df_data.fillna('N3', inplace=True)
    
    return df_data

# Generate Data

In [86]:
prod_id1 = 'B003ZDX2RW'
prod_id2 = 'B00503PKV6'
date = np.datetime64('2011-12-10')

In [101]:
df_data = total_return_product_times(df_cmp_inv, df_tran_re, prod_id1, prod_id2)

In [102]:
df_data = quantity_purchased(df_data, df_tran_re, prod_id1, prod_id2, '6M')

In [103]:
df_data = recenvy(df_data, df_tran_re, prod_id1, prod_id2, date)

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 'invoicedate' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: 'invoicedate' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


In [104]:
df_data = num_invoices_last_k_months(df_tran, df_data, prod_id1, prod_id2, 6)

In [105]:
df_trans = pd.read_sql('SELECT transaction_info.*, product.category FROM transaction_info JOIN product ON product.productid = transaction_info.productid WHERE category = \'Dining and Entertaining\' AND sales > 0 AND customerid != \'GUEST\'' , connect)
df_data = avg_bud_for_cat_in_k_months(df_trans, df_data, 6)

In [106]:
df_data

,customerid,class,RT_return,RT_quantity_LT,Recency,Num_inv_6_months,avg_bud_6_months
0,12359,C,RE,Q2,R2,N3,Poor
1,12362,A,RE,Q1,R1,N1,Poor
2,12415,D,RG,Q5,R1,N5,Rich
3,12417,A,RE,Q1,R1,N1,Poor
4,12502,E,RE,Q5,R3,N5,Poor
5,12539,D,RE,Q2,R2,N3,Rich
6,12621,D,RG,Q3,R1,N4,Poor
7,12637,A,RE,Q1,R1,N1,Poor
8,12700,E,RE,Q5,R3,N5,Poor
9,12731,C,RE,Q2,R3,N3,Rich


In [108]:
df_data.to_csv("data_loyalty.csv", index_label=False)